In [1]:
import os
from PIL import Image
from transformers import AutoModelForCausalLM
from deepseek_vl2.models import DeepseekVLV2Processor, DeepseekVLV2ForCausalLM
from deepseek_vl2.utils.io import load_pil_images
import torch
import gc
import traceback
from accelerate import init_empty_weights, load_checkpoint_and_dispatch
from torch.cuda.amp import autocast

/home/elboardy/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/elboardy/.local/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/elboardy/.local/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
cases_dir = '/media/RLAB-Disk01/(final)merged_images_with_labels_order_and_folders_mask_normalized/'
cache_dir = '/media/RLAB-Disk01/Large-Language-Models-Weights'
model_id = "deepseek-ai/deepseek-vl2"
offload_folder = '/media/RLAB-Disk01/Large-Language-Models-Weights'

In [3]:
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'
# torch.cuda.set_per_process_memory_fraction(0.9)
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'


In [4]:
# torch.backends.cuda.matmul.allow_tf32 = True
# torch.backends.cudnn.allow_tf32 = True
# torch.set_float32_matmul_precision('medium')
# os.environ['PYTORCH_CUDA_ALLOC_CONF']="expandable_segments:True"

In [5]:
# import os

# # Make sure cases_dir is properly defined
# # cases_dir = "your/cases/directory/path"

# deleted_count = 0
# for case in os.listdir(cases_dir):
#     case_dir = os.path.join(cases_dir, case)
#     if os.path.isdir(case_dir):
#         response_path = os.path.join(case_dir, 'deepseek-vl-small.txt')
#         if os.path.exists(response_path):
#             try:
#                 os.remove(response_path)
#                 deleted_count += 1
#                 print(f"Deleted: {response_path}")
#             except Exception as e:
#                 print(f"Error deleting {response_path}: {str(e)}")

# print(f"\nDeleted {deleted_count} response files.")
# print(f"Remaining cases without response file: {len(os.listdir(cases_dir)) - deleted_count}")

In [6]:
cases_to_process = []
for case in os.listdir(cases_dir):
    case_dir = os.path.join(cases_dir, case)
    if os.path.isdir(case_dir):
        response_path = os.path.join(case_dir, 'deepseek-vl2.txt')
        if not os.path.exists(response_path):
            cases_to_process.append(case)
print(f"Found {len(cases_to_process)} cases to process.")


Found 40 cases to process.


In [7]:

torch.set_default_tensor_type(torch.FloatTensor) 

processor = DeepseekVLV2Processor.from_pretrained(model_id, torch_dtype=torch.float16,
                                                 device_map="cpu", trust_remote_code=True, 
                                                 cache_dir=cache_dir, 
                                                 offload_folder=offload_folder)
tokenizer = processor.tokenizer
model: DeepseekVLV2ForCausalLM = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="cpu",
    trust_remote_code=True,
    use_safetensors=True,
    cache_dir=cache_dir,
    offload_folder=offload_folder,
    # attn_implementation="eager",
    
)

# model.to(torch.float32)


# model = model.eval()


/home/elboardy/.local/lib/python3.9/site-packages/torch/__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:432.)
  _C._set_default_tensor_type(t)
/home/elboardy/.local/lib/python3.9/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Add pad token = ['<｜▁pad▁｜>'] to the tokenizer
<｜▁pad▁｜>:2
Add image token = ['<image>'] to the tokenizer
<image>:128815
Add grounding-related tokens = ['<|ref|>', '<|/ref|>', '<|det|>', '<|/det|>', '<|grounding|>'] to the tokenizer with input_ids
<|ref|>:128816
<|/ref|>:128817
<|det|>:128818
<|/det|>:128819
<|grounding|>:128820
Add chat tokens = ['<|User|>', '<|Assistant|>'] to the tokenizer with input_ids
<|User|>:128821
<|Assistant|>:128822



Loading checkpoint shards: 100%|██████████| 8/8 [00:05<00:00,  1.39it/s]


In [8]:
print(next(model.parameters()).dtype) 

torch.float16


In [9]:
def build_conversation(case_dir, user_prompt,system_prompt):
    image_files = [f for f in os.listdir(case_dir) if f.lower().endswith('.png')]
    image_paths = [os.path.join(case_dir, f) for f in image_files]
    
    image_files= [image_files[-1]]
    image_paths = [image_paths[-1]]
    # Create image placeholders
    print(image_files)
    print(image_paths)
    image_tags = " ".join(["<image>"] * len(image_files))
    print(image_tags)
    
    return [

         {
            "role": "<|User|>",
            "content": f"{image_tags}\n<|ref|>{system_prompt}\n{user_prompt}<|/ref|>",
            "images": image_paths
        },

        {"role": "<|Assistant|>", "content": ""}
    ]


In [10]:
# failed_cases = []


# for case in cases_to_process:
#     case_dir = os.path.join(cases_dir, case)
#     try:
#         # Load clinical info
#         clinical_path = os.path.join(case_dir, 'diagnostic_prompt.txt')
#         if not os.path.exists(clinical_path):
#             print(f"Missing clinical info for {case}")
#             failed_cases.append(case)
#             continue
            
#         clinical_info = open(clinical_path).read()
        

#         system_prompt = """Consider that you are a professional radiologist with several years of experience and you are now treating a patient. Write a fully detailed diagnosis report for this case, avoiding any potential hallucination and paying close attention to all of the batch images attached to this message.

# Use the following structure for the report:

# ## Radiologist's Report

# ### Patient Information:
# - *Age:* 65
# - *Sex:* Male
# - *Days from earliest imaging to surgery:* 1
# - *Histopathological Subtype:* Glioblastoma
# - *WHO Grade:* 4
# - *IDH Status:* Mutant
# - *Preoperative KPS:* 80
# - *Preoperative Contrast-Enhancing Tumor Volume (cm³):* 103.21
# - *Preoperative T2/FLAIR Abnormality (cm³):* 36.29
# - *Extent of Resection (EOR %):* 100.0
# - *EOR Type:* Gross Total Resection (GTR)
# - *Adjuvant Therapy:* Radiotherapy (RT) + Temozolomide (TMZ)
# - *Progression-Free Survival (PFS) Days:* 649
# - *Overall Survival (OS) Days:* 736

# #### Tumor Characteristics:

# #### Segmentation Analysis:

# #### Surgical Considerations:

# ### Clinical Summary:

# ### Recommendations:

# ### Prognostic Considerations:

# ### Follow-Up Plan:

# ### Additional Notes*(if any)*:

# Ensure all findings from all of the images and clinical data provided. Please mention at the end of the report how many images were reviewed."""

#         user_prompt = f"""You will be given batches of images, which are different sequences of MRI scans. 
#     The images are for patients who are likely to have a brain tumor. Each image will contain up to 10 slices for 5 different sequences and the segmentation masks for the tumor at the bottom row of the image. 
#     Additional clinical data about the patient is: 
#     {clinical_info}"""



#         conversation = build_conversation(case_dir, user_prompt,system_prompt)
#         pil_images = load_pil_images(conversation)
        
#         conversation = build_conversation(case_dir, user_prompt,system_prompt)
#         pil_images = load_pil_images(conversation)
        
#         print(conversation)

#         # Process inputs
#         inputs = processor(
#             conversations=conversation,
#             images=pil_images,
#             force_batchify=True,
#             system_prompt=system_prompt
#         ).to(model.device)

#         # Generate embeddings
#         inputs_embeds = model.prepare_inputs_embeds(**inputs)

#         print(f"Messages prepared for case {case}")
#         # Generate response



#         outputs = model.language.generate(
#             inputs_embeds=inputs_embeds,
#             attention_mask=inputs.attention_mask,
#             pad_token_id=tokenizer.eos_token_id,
#             bos_token_id=tokenizer.bos_token_id,
#             eos_token_id=tokenizer.eos_token_id,
#             max_new_tokens=4096,
#             temperature=0.7,
#             top_p=0.9,
#             do_sample=False,
#             use_cache=True,
#         )
        
#         # Decode and save response

#         print(outputs.cpu().tolist())  
#         response = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)

#         print(f"Response generated for case {case}, length: {len(response)} characters")

#         print(response)

#         del inputs, outputs, pil_images, conversation, inputs_embeds
#         gc.collect()
#         torch.cuda.empty_cache()
#         model._clear_cuda_cache()

#         with open(os.path.join(case_dir, 'deepseek-vl2.txt'), 'w', encoding='utf-8') as f:
#             f.write(response.strip())

#         print("Response saved.",case)  
        
#     except Exception as e:
#         error_msg = f"\n\nError processing {case}:\n{traceback.format_exc()}"
#         print(error_msg)
#         failed_cases.append(case)

# # Save failed cases
# if failed_cases:
#     with open(os.path.join(cases_dir, 'failed_deepseek-vl2.txt.txt'), 'w') as f:
#         f.write("\n".join(failed_cases))

In [13]:
failed_cases = []


for case in cases_to_process:
    case_dir = os.path.join(cases_dir, case)

    clinical_path = os.path.join(case_dir, 'diagnostic_prompt.txt')
    if not os.path.exists(clinical_path):
            print(f"Missing clinical info for {case}")
            failed_cases.append(case)
            continue
            
    clinical_info = open(clinical_path).read()
        

    system_prompt = """Consider that you are a professional radiologist with several years of experience and you are now treating a patient. Write a fully detailed diagnosis report for this case, avoiding any potential hallucination and paying close attention to all of the batch images attached to this message.

Use the following structure for the report:

## Radiologist's Report

### Patient Information:
- *Age:* 65
- *Sex:* Male
- *Days from earliest imaging to surgery:* 1
- *Histopathological Subtype:* Glioblastoma
- *WHO Grade:* 4
- *IDH Status:* Mutant
- *Preoperative KPS:* 80
- *Preoperative Contrast-Enhancing Tumor Volume (cm³):* 103.21
- *Preoperative T2/FLAIR Abnormality (cm³):* 36.29
- *Extent of Resection (EOR %):* 100.0
- *EOR Type:* Gross Total Resection (GTR)
- *Adjuvant Therapy:* Radiotherapy (RT) + Temozolomide (TMZ)
- *Progression-Free Survival (PFS) Days:* 649
- *Overall Survival (OS) Days:* 736

#### Tumor Characteristics:

#### Segmentation Analysis:

#### Surgical Considerations:

### Clinical Summary:

### Recommendations:

### Prognostic Considerations:

### Follow-Up Plan:

### Additional Notes*(if any)*:

Ensure all findings from all of the images and clinical data provided. Please mention at the end of the report how many images were reviewed."""

    user_prompt = f"""You will be given batches of images, which are different sequences of MRI scans. 
    The images are for patients who are likely to have a brain tumor. Each image will contain up to 10 slices for 5 different sequences and the segmentation masks for the tumor at the bottom row of the image. 
    Additional clinical data about the patient is: 
    {clinical_info}"""



        # Prepare conversation
    conversation = build_conversation(case_dir, user_prompt,system_prompt)
    pil_images = load_pil_images(conversation)
        
    print(conversation)
    
    print("-----------------------------------------------------------------------------------------------")
        # Process inputs
    inputs = processor(
            conversations=conversation,
            images=pil_images,
            force_batchify=True,
            system_prompt=system_prompt
        ).to(model.device,dtype=torch.float16)
    
    print(inputs)
    print(type(inputs))

    print("-----------------------------------------------------------------------------------------------")
    
    model = model.eval()
    model = model.to("cuda")

    inputs_embeds = model.prepare_inputs_embeds(**inputs)

    print(f"Messages prepared for case {case}")
        # Generate response



    outputs = model.language.generate(
            inputs_embeds=inputs_embeds,
            attention_mask=inputs.attention_mask,
            pad_token_id=tokenizer.eos_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            max_new_tokens=4096,
            temperature=0.7,
            top_p=0.9,
            do_sample=False,
            use_cache=True,
        )
        
        # Decode and save response

    print(outputs.cpu().tolist())  
    response = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)

    print(f"Response generated for case {case}, length: {len(response)} characters")

    print(response)

    del inputs, outputs, pil_images, conversation, inputs_embeds
    gc.collect()
    torch.cuda.empty_cache()
    model._clear_cuda_cache()

    with open(os.path.join(case_dir, 'deepseek-vl.txt'), 'w', encoding='utf-8') as f:
            f.write(response.strip())

    print("Response saved.",case)        
        # Cleanup


    
        
    # except Exception as e:
    #     error_msg = f"\n\nError processing {case}:\n{traceback.format_exc()}"
    #     print(error_msg)
    #     failed_cases.append(case)

# Save failed cases
if failed_cases:
    with open(os.path.join(cases_dir, 'failed_deepseek-vl-small.txt'), 'w') as f:
        f.write("\n".join(failed_cases))

['RHUH-0019_batch_9.png']
['/media/RLAB-Disk01/(final)merged_images_with_labels_order_and_folders_mask_normalized/RHUH-0019/RHUH-0019_batch_9.png']
<image>
[{'role': '<|User|>', 'content': "<image>\n<|ref|>Consider that you are a professional radiologist with several years of experience and you are now treating a patient. Write a fully detailed diagnosis report for this case, avoiding any potential hallucination and paying close attention to all of the batch images attached to this message.\n\nUse the following structure for the report:\n\n## Radiologist's Report\n\n### Patient Information:\n- *Age:* 65\n- *Sex:* Male\n- *Days from earliest imaging to surgery:* 1\n- *Histopathological Subtype:* Glioblastoma\n- *WHO Grade:* 4\n- *IDH Status:* Mutant\n- *Preoperative KPS:* 80\n- *Preoperative Contrast-Enhancing Tumor Volume (cm³):* 103.21\n- *Preoperative T2/FLAIR Abnormality (cm³):* 36.29\n- *Extent of Resection (EOR %):* 100.0\n- *EOR Type:* Gross Total Resection (GTR)\n- *Adjuvant The

OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacity of 47.50 GiB of which 55.75 MiB is free. Process 4240 has 430.89 MiB memory in use. Including non-PyTorch memory, this process has 46.41 GiB memory in use. Of the allocated memory 35.44 GiB is allocated by PyTorch, and 10.56 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)